In [1]:
"""
Diffie-Hellman Key Exchange with Man-in-the-Middle Attack Simulation
Demonstrates how an attacker can intercept and manipulate key exchanges
"""

import random

class DiffieHellmanParticipant:
    """Represents a participant in the Diffie-Hellman key exchange"""

    def __init__(self, name, prime, generator):
        self.name = name
        self.prime = prime
        self.generator = generator
        self.private_key = random.randint(2, prime - 2)
        self.public_key = None
        self.shared_secret = None

    def generate_public_key(self):
        """Generate public key: g^private mod p"""
        self.public_key = pow(self.generator, self.private_key, self.prime)
        return self.public_key

    def compute_shared_secret(self, other_public_key):
        """Compute shared secret: other_public^private mod p"""
        self.shared_secret = pow(other_public_key, self.private_key, self.prime)
        return self.shared_secret


class Attacker:
    """Represents the Man-in-the-Middle attacker"""

    def __init__(self, name, prime, generator):
        self.name = name
        self.prime = prime
        self.generator = generator
        # Attacker creates two sets of keys for each victim
        self.private_key_for_alice = random.randint(2, prime - 2)
        self.private_key_for_bob = random.randint(2, prime - 2)
        self.public_key_for_alice = None
        self.public_key_for_bob = None
        self.shared_secret_with_alice = None
        self.shared_secret_with_bob = None

    def generate_public_keys(self):
        """Generate two public keys to impersonate both parties"""
        self.public_key_for_alice = pow(self.generator, self.private_key_for_alice, self.prime)
        self.public_key_for_bob = pow(self.generator, self.private_key_for_bob, self.prime)
        return self.public_key_for_alice, self.public_key_for_bob

    def intercept_and_compute_secrets(self, alice_public, bob_public):
        """Intercept public keys and compute shared secrets with both parties"""
        # Compute shared secret with Alice using Alice's public key
        self.shared_secret_with_alice = pow(alice_public, self.private_key_for_alice, self.prime)
        # Compute shared secret with Bob using Bob's public key
        self.shared_secret_with_bob = pow(bob_public, self.private_key_for_bob, self.prime)
        return self.shared_secret_with_alice, self.shared_secret_with_bob


def print_separator(title=""):
    """Print a visual separator"""
    print("\n" + "="*70)
    if title:
        print(f"  {title}")
        print("="*70)


def simulate_normal_diffie_hellman():
    """Simulate normal Diffie-Hellman key exchange without attack"""
    print_separator("NORMAL DIFFIE-HELLMAN KEY EXCHANGE (No Attack)")

    # Public parameters (known to everyone)
    prime = 23  # Small prime for demonstration
    generator = 5

    print(f"\nPublic Parameters:")
    print(f"  Prime (p) = {prime}")
    print(f"  Generator (g) = {generator}")

    # Create Alice and Bob
    alice = DiffieHellmanParticipant("Alice", prime, generator)
    bob = DiffieHellmanParticipant("Bob", prime, generator)

    print(f"\nPrivate Keys (Secret):")
    print(f"  Alice's private key = {alice.private_key}")
    print(f"  Bob's private key = {bob.private_key}")

    # Generate public keys
    alice_public = alice.generate_public_key()
    bob_public = bob.generate_public_key()

    print(f"\nPublic Keys (Exchanged Openly):")
    print(f"  Alice's public key = {alice_public}")
    print(f"  Bob's public key = {bob_public}")

    # Compute shared secrets
    alice_shared = alice.compute_shared_secret(bob_public)
    bob_shared = bob.compute_shared_secret(alice_public)

    print(f"\nShared Secrets:")
    print(f"  Alice computed = {alice_shared}")
    print(f"  Bob computed = {bob_shared}")
    print(f"\n✓ Keys Match: {alice_shared == bob_shared}")


def simulate_mitm_attack():
    """Simulate Man-in-the-Middle attack on Diffie-Hellman"""
    print_separator("MAN-IN-THE-MIDDLE ATTACK SIMULATION")

    # Public parameters
    prime = 23
    generator = 5

    print(f"\nPublic Parameters:")
    print(f"  Prime (p) = {prime}")
    print(f"  Generator (g) = {generator}")

    # Create participants
    alice = DiffieHellmanParticipant("Alice", prime, generator)
    bob = DiffieHellmanParticipant("Bob", prime, generator)
    tom = Attacker("Tom (Attacker)", prime, generator)

    print(f"\nPrivate Keys:")
    print(f"  Alice's private key = {alice.private_key}")
    print(f"  Bob's private key = {bob.private_key}")
    print(f"  Tom's private key (for Alice) = {tom.private_key_for_alice}")
    print(f"  Tom's private key (for Bob) = {tom.private_key_for_bob}")

    # Step 1: Alice and Bob generate their public keys
    alice_public = alice.generate_public_key()
    bob_public = bob.generate_public_key()

    print(f"\n--- Step 1: Public Key Generation ---")
    print(f"  Alice generates public key = {alice_public}")
    print(f"  Bob generates public key = {bob_public}")

    # Step 2: Tom intercepts and generates fake public keys
    tom_public_for_alice, tom_public_for_bob = tom.generate_public_keys()

    print(f"\n--- Step 2: Tom's Interception ---")
    print(f"  Tom intercepts Alice's public key: {alice_public}")
    print(f"  Tom intercepts Bob's public key: {bob_public}")
    print(f"  Tom generates fake public key for Alice: {tom_public_for_alice}")
    print(f"  Tom generates fake public key for Bob: {tom_public_for_bob}")

    # Step 3: Tom forwards fake keys
    print(f"\n--- Step 3: Tom Forwards Fake Keys ---")
    print(f"  Tom sends {tom_public_for_bob} to Alice (pretending it's from Bob)")
    print(f"  Tom sends {tom_public_for_alice} to Bob (pretending it's from Alice)")

    # Step 4: Alice and Bob compute "shared" secrets (actually with Tom)
    alice_shared = alice.compute_shared_secret(tom_public_for_bob)  # Alice thinks she's sharing with Bob
    bob_shared = bob.compute_shared_secret(tom_public_for_alice)    # Bob thinks he's sharing with Alice

    # Step 5: Tom computes shared secrets with both
    tom.intercept_and_compute_secrets(alice_public, bob_public)

    print(f"\n--- Step 4: Shared Secret Computation ---")
    print(f"  Alice computes shared secret = {alice_shared}")
    print(f"  Bob computes shared secret = {bob_shared}")
    print(f"  Tom computes shared secret with Alice = {tom.shared_secret_with_alice}")
    print(f"  Tom computes shared secret with Bob = {tom.shared_secret_with_bob}")

    # Verification
    print_separator("ATTACK RESULTS")
    print(f"\n✓ Alice's key matches Tom's key for Alice: {alice_shared == tom.shared_secret_with_alice}")
    print(f"  Alice's key = {alice_shared}")
    print(f"  Tom's key (Alice side) = {tom.shared_secret_with_alice}")

    print(f"\n✓ Bob's key matches Tom's key for Bob: {bob_shared == tom.shared_secret_with_bob}")
    print(f"  Bob's key = {bob_shared}")
    print(f"  Tom's key (Bob side) = {tom.shared_secret_with_bob}")

    print(f"\n✗ Alice and Bob DO NOT share the same key: {alice_shared != bob_shared}")
    print(f"  Alice's key = {alice_shared}")
    print(f"  Bob's key = {bob_shared}")

    print(f"\n⚠️  ATTACK SUCCESSFUL!")
    print(f"  Tom can decrypt messages from Alice using key: {tom.shared_secret_with_alice}")
    print(f"  Tom can decrypt messages from Bob using key: {tom.shared_secret_with_bob}")
    print(f"  Tom can read, modify, and re-encrypt all communications!")


def main():
    """Main function to run the demonstration"""
    print("\n" + "="*70)
    print("  DIFFIE-HELLMAN KEY EXCHANGE & MITM ATTACK DEMONSTRATION")
    print("="*70)

    # First show normal exchange
    simulate_normal_diffie_hellman()

    print("\n" * 2)

    # Then show MITM attack
    simulate_mitm_attack()

    print("\n" + "="*70)
    print("  CONCLUSION")
    print("="*70)
    print("\nThe Man-in-the-Middle attack succeeds because:")
    print("1. Diffie-Hellman has no built-in authentication")
    print("2. Tom can impersonate both Alice and Bob")
    print("3. Alice and Bob cannot verify the source of public keys")
    print("\nDefense: Use authenticated key exchange (e.g., digital signatures)")
    print("="*70 + "\n")


if __name__ == "__main__":
    main()


  DIFFIE-HELLMAN KEY EXCHANGE & MITM ATTACK DEMONSTRATION

  NORMAL DIFFIE-HELLMAN KEY EXCHANGE (No Attack)

Public Parameters:
  Prime (p) = 23
  Generator (g) = 5

Private Keys (Secret):
  Alice's private key = 16
  Bob's private key = 18

Public Keys (Exchanged Openly):
  Alice's public key = 3
  Bob's public key = 6

Shared Secrets:
  Alice computed = 2
  Bob computed = 2

✓ Keys Match: True




  MAN-IN-THE-MIDDLE ATTACK SIMULATION

Public Parameters:
  Prime (p) = 23
  Generator (g) = 5

Private Keys:
  Alice's private key = 15
  Bob's private key = 19
  Tom's private key (for Alice) = 4
  Tom's private key (for Bob) = 3

--- Step 1: Public Key Generation ---
  Alice generates public key = 19
  Bob generates public key = 7

--- Step 2: Tom's Interception ---
  Tom intercepts Alice's public key: 19
  Tom intercepts Bob's public key: 7
  Tom generates fake public key for Alice: 4
  Tom generates fake public key for Bob: 10

--- Step 3: Tom Forwards Fake Keys ---
  Tom sends 10 to 